<a href="https://colab.research.google.com/github/rze7ud/DS3001_Final_Project/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

repo_url = f"https://{GITHUB_TOKEN}@github.com/rze7ud/DS3001_Final_Project/"

os.system(f"git clone {repo_url}")

0

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
df = pd.read_csv("/content/DS3001_Final_Project/data/clemson_2001_2024.csv")
df['season'] = df['season'].astype(int)
df_opp = pd.read_csv("/content/DS3001_Final_Project/data/clemson_opp_2001_2024.csv")

In [10]:
df_combined = pd.concat([df, df_opp], axis=1)

In [22]:
out = pd.merge_asof(df_opp, df, on ='season')

In [23]:
print(out)

     season  week   home    win  clemson_points             opp  opp_points  \
0      2001     2   True   True            21.0             UCF        13.0   
1      2001     5   True  False            24.0        Virginia        26.0   
2      2001     6  False   True            47.0    Georgia Tech        44.0   
3      2001     8  False   True            45.0        NC State        37.0   
4      2001     9   True  False             3.0  North Carolina        38.0   
..      ...   ...    ...    ...             ...             ...         ...   
263    2024    10   True  False            21.0      Louisville        33.0   
264    2024    11  False   True            24.0   Virginia Tech        14.0   
265    2024    12  False   True            24.0      Pittsburgh        20.0   
266    2024    14   True  False            14.0  South Carolina        17.0   
267    2024    15  False    NaN             NaN             SMU         NaN   

    opp_conference  opp_def_explosiveness  opp_def_